In [157]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


In [158]:
import random

# Helper functions as provided
def valid_row(sudoku_board, row):
    assigned = []
    for value in sudoku_board[row]:
        if value != 0:
            if value in assigned:
                return False
            else:
                assigned.append(value)
    return True

def valid_column(sudoku_board, column):
    assigned = []
    for row in sudoku_board:
        value = row[column]
        if value != 0:
            if value in assigned:
                return False
            else:
                assigned.append(value)
    return True

def valid_quadrant(sudoku_board, quadrant):
    starting_row = (quadrant // 3) * 3
    starting_column = (quadrant % 3) * 3
    assigned = []
    for row in sudoku_board[starting_row : starting_row + 3]:
        for value in row[starting_column : starting_column + 3]:
            if value != 0:
                if value in assigned:
                    return False
                assigned.append(value)
    return True

def complete_solution(sudoku_board):
    for row in sudoku_board:
        for value in row:
            if value == 0:
                return False
    return True

def valid_solution(sudoku_board):
    for i in range(9):
        if not valid_row(sudoku_board, i):
            return False
        if not valid_column(sudoku_board, i):
            return False
        if not valid_quadrant(sudoku_board, i):
            return False
    return True

def check_solution(puzzle, solution):
    for row in range(9):
        for col in range(9):
            if puzzle[row][col] != 0 and puzzle[row][col] != solution[row][col]:
                return 'Solution is not valid, changes a fixed value in the puzzle'
    if not valid_solution(solution):
        return 'Solution does not respect the constraints'
    if not complete_solution(solution):
        return 'Solution is only partial'
    return 'Solution is valid'

def pretty_print(sudoku_board):
    for row in sudoku_board:
        print(" ".join(map(str, row)))

def sudoku_string_to_matrix(sudoku):
    sudoku_board = []
    for i in range(9):
        starting_row = i * 9
        row = []
        for j in range(9):
            row.append(int(sudoku[starting_row + j]))
        sudoku_board.append(row)
    return sudoku_board

def sudoku_matrix_to_string(sudoku_board):
    return ''.join(map(lambda row: ''.join(map(str, row)), sudoku_board))

def random_initialize_sudoku(puzzle):
    sudoku_board = sudoku_string_to_matrix(puzzle)
    for i in range(9):
        for j in range(9):
            if sudoku_board[i][j] == 0:
                sudoku_board[i][j] = random.randint(1, 9)
    return sudoku_board

def count_conflicts(sudoku_board, row, col):
    conflicts = 0
    value = sudoku_board[row][col]

    for i in range(9):
        if i != col and sudoku_board[row][i] == value:
            conflicts += 1

    for i in range(9):
        if i != row and sudoku_board[i][col] == value:
            conflicts += 1

    starting_row = (row // 3) * 3
    starting_column = (col // 3) * 3
    for i in range(3):
        for j in range(3):
            if (starting_row + i) != row and (starting_column + j) != col and sudoku_board[starting_row + i][starting_column + j] == value:
                conflicts += 1

    return conflicts

# Test the helper functions with a sample Sudoku string
sudoku_string = '000005000037800006200167800300009720905783060470200089890502600540900000000400593'
#sudoku_string2 = '003709008108346957976810004009063785380157692567080340890230570632578419005691023'
initialized_sudoku = random_initialize_sudoku(sudoku_string)
pretty_print(initialized_sudoku)
print("\nConflicts at (0, 0):", count_conflicts(initialized_sudoku, 0, 0))
print("\nCheck if the solution is valid:", check_solution(sudoku_string_to_matrix(sudoku_string), initialized_sudoku))


5 6 9 2 3 5 5 6 4
8 3 7 8 9 5 5 9 6
2 5 2 1 6 7 8 5 4
3 2 8 9 9 9 7 2 7
9 7 5 7 8 3 8 6 8
4 7 2 2 2 7 3 8 9
8 9 3 5 3 2 6 8 4
5 4 2 9 7 2 1 4 4
6 5 6 4 7 2 5 9 3

Conflicts at (0, 0): 4

Check if the solution is valid: Solution does not respect the constraints


In [161]:
import random
from copy import deepcopy

def find_best_number(current_state, row, col): #takes the current state of the Sudoku board, and the row and column of the cell to be improved, and returns the best possible number for that cell
    min_conflicts = count_conflicts(current_state, row, col)
    number_candidate = []
    
    for num in range(1, 10):
        temp = deepcopy(current_state)
        temp[row][col] = num
        current_conflicts = count_conflicts(temp, row, col)
        if current_conflicts < min_conflicts:
            number_candidate = [num]  # Reset the list with a better candidate
            min_conflicts = current_conflicts
        elif current_conflicts == min_conflicts:
            number_candidate.append(num)  # Add to list of candidates with equal conflict count

    if number_candidate:
        return number_candidate[random.randint(0, len(number_candidate) - 1)]  # Return a random choice among candidates
    else:
        return current_state[row][col]  # Return the original number if no better option was found

def min_conflicts(board_string):
    # initalizations
    board = sudoku_string_to_matrix(board_string)
    current_state = random_initialize_sudoku(board_string)
    changeable_cells = [(i, j) for i in range(9) for j in range(9) if board[i][j] == 0]

    # Instead of doing max iterations as a function parameter, I will restart if there is no improvement after a certain number of iterations so it keeps trying to find a solution
    steps = 0
    max_steps = 5000

    while not valid_solution(current_state):
        conflicts = deepcopy(changeable_cells)

        for row, col in conflicts:
            # Find the best number (fewer conflicts) for the cell and update the state
            better_num = find_best_number(current_state, row, col)
            current_state[row][col] = better_num 

            
            min_conflicts_count = count_conflicts(current_state, row, col) 
            # reset the steps if there is improvement
            if min_conflicts_count < count_conflicts(current_state, row, col):
                steps = 0
                break
            else:
                steps += 1
        
        if steps > max_steps:
            current_state = random_initialize_sudoku(board_string)
            steps = 0

    return current_state



In [163]:
# Test to server
problem = get_problem('easy-sudoku')

pprint.pprint(problem)

sudoku_board = problem['sudoku']

print('Input puzzle')
pretty_print(sudoku_string_to_matrix(sudoku_board))
print('\n')

# Solve the puzzle
solved_sudoku = min_conflicts(sudoku_board)
print('Solved puzzle')
pretty_print(solved_sudoku)
check_solution(sudoku_string_to_matrix(sudoku_board), solved_sudoku)

answer = {
    'sudoku': sudoku_matrix_to_string(solved_sudoku),
    'id': problem['id']
}

submit_answer('sudoku', answer)


{'id': 4,
 'sudoku': '000005000037800006200167800300009720905783060470200089890502600540900000000400593'}
Input puzzle
0 0 0 0 0 5 0 0 0
0 3 7 8 0 0 0 0 6
2 0 0 1 6 7 8 0 0
3 0 0 0 0 9 7 2 0
9 0 5 7 8 3 0 6 0
4 7 0 2 0 0 0 8 9
8 9 0 5 0 2 6 0 0
5 4 0 9 0 0 0 0 0
0 0 0 4 0 0 5 9 3


Solved puzzle
6 8 4 3 9 5 1 7 2
1 3 7 8 2 4 9 5 6
2 5 9 1 6 7 8 3 4
3 1 8 6 4 9 7 2 5
9 2 5 7 8 3 4 6 1
4 7 6 2 5 1 3 8 9
8 9 1 5 3 2 6 4 7
5 4 3 9 7 6 2 1 8
7 6 2 4 1 8 5 9 3
PASSED
